# KDD99 Deep Nueral Network Intrusion Detection

### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
from IPython.display import clear_output as clr
from sklearn.utils import class_weight
%matplotlib inline

In [2]:
import keras as kk
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko
import keras.regularizers as kr
from sklearn.model_selection import train_test_split as tts

Using TensorFlow backend.


In [3]:
import tensorflow as tf

In [4]:
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.preprocessing import LabelEncoder as LE

### Setting working Directory

In [5]:
dir_path = (os.getcwd()+'\\').replace("\\","/")
print("Working Directory : ", dir_path)

Working Directory :  /media/abhikcr/New Volume/Study/M.tech 1st Sem/Cyber Security/Homeworks/KDD/extracted/


### Fucntions to Load files..

In [6]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [7]:
def readlines(fname, start, end):
#     fname = 'kddcup.data.corrected'
    file = open(fname)
    batch = []
    r = end - start
    for i in range(start):
        line = file.readline()
    for i in range(r):
        line = file.readline()
        batch.append(line.split(','))
    file.close()
    return batch

In [8]:
def rmreadlines(fname, size, ntypes):
#     fname = 'kddcup.data.corrected'
    Nobs = file_len(fname)
    choices = np.sort(np.random.choice(Nobs,size, replace=False))
    flag = False
    k = 0
    
    file = open(fname)
    batch = []
    uniq = []
    
    for i in range(Nobs):

        line = file.readline()
        var = line.split(',')
        size = len(var)
        val = var[size-1]
        
        if(val not in uniq):
            batch.append(var)
            uniq.append(val)
            if(len(uniq) == ntypes):
                flag = True
        
        if(k<choices.shape[0]):
            if(choices[k] == i):
                batch.append(line.split(','))
                k = k+1
            else:
                pass
        else:
            if(flag):
                break;
    
    file.close()
    return batch

## Preprocessing Splitting data in train and test files

In [9]:
def split_file(fname, trainname, testname):
    Nobs = file_len(fname)
    tfp = int(Nobs*0.25)
    choices = np.sort(np.random.choice(Nobs,tfp, replace=False))
    k = 0
    file = open(fname)
    filetr = open(trainname, 'w')
    filets = open(testname, 'w')
    
    uniq_pro = []
    uniq_srv = []
    uniq_flg = []
    
    uniq = []
    
    for i in range(Nobs):
        
        line = file.readline()
        
        var = line.split(',')
        arr_small  = var[1:4]
        
        
        size = len(var)
        val = var[size-1]
        
        if(val in uniq):
            pass
        else:
            filets.write(line)
            uniq.append(val)
            
        if(arr_small[0] not in uniq_pro):
            uniq_pro.append(arr_small[0])
            
        if(arr_small[1] not in uniq_srv):
            uniq_srv.append(arr_small[1])
            
        if(arr_small[2] not in uniq_flg):
            uniq_flg.append(arr_small[2])
        
        
        if(k<choices.shape[0]):
            if(choices[k] == i):
                filets.write(line)
                k = k+1
            else:
                filetr.write(line)
        else:
            filetr.write(line)
            
    filets.close()
    filetr.close()
    file.close()
    return Nobs, choices.shape[0], [uniq_pro,uniq_srv, uniq_flg]

In [10]:
# Already spliited dont repeat...
split = False
if(split):
    Nobs, test_size, cat_uniqs = split_file('kddcup.data.corrected', 'train','test')

In [11]:
# np.save('uniqs',cat_uniqs)
cat_uniqs = list(np.load('uniqs.npy'))
cat_uniqs[0]

['tcp', 'udp', 'icmp']

### Loading trainning data batchwise...

In [12]:
def myArrayConverter(arr):
    '''
    To convert string to floats
    '''
    convertArr = []
    for s in arr.ravel():    
        try:
            value = float(s)
        except ValueError:
            value = s
        convertArr.append(value)
    return array(convertArr,dtype=object).reshape(arr.shape)

In [13]:
# file names
trainfile = 'train_red.npy'
testfile = 'test_red.npy'

### Bacth Processing...

Function to convert categorical Y values to one_hot vector

In [14]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

Creating label ids

In [15]:
label_list = pd.read_csv('training_attack_types', delimiter = ' ', index_col = False, names =['type', 'category'])
label_list = label_list.assign(id_type=(label_list['type']).astype('category').cat.codes)
label_list = label_list.assign(id_cat=(label_list['category']).astype('category').cat.codes)
label_list = label_list.append({'type':'normal', 'category':'normal','id_type':'-1','id_cat':'-1'}, ignore_index = True)
label_list['id_type'] = label_list['id_type'].astype(int)+1
label_list['id_cat'] = label_list['id_cat'].astype(int)+1

In [16]:
nClasses = label_list.shape[0]
label_list

,type,category,id_type,id_cat
0,back,dos,1,1
1,buffer_overflow,u2r,2,4
2,ftp_write,r2l,3,3
3,guess_passwd,r2l,4,3
4,imap,r2l,5,3
5,ipsweep,probe,6,2
6,land,dos,7,1
7,loadmodule,u2r,8,4
8,multihop,r2l,9,3
9,neptune,dos,10,1


In [17]:
import gc

In [18]:
# Sample batching....
#currpos = 0
# batch = readlines(trainfile, currpos, currpos + batch_size)
batcharr = np.load(trainfile)

In [19]:
batcharr.shape

(833529, 42)

In [20]:
X = batcharr[:,:-1]

Y = batcharr[:,-1]
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()

for i in label_list['type']:

    Y_cat[Y == i + "."] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+ "."] = label_list[label_list['type'] == i]['id_type']


In [21]:
# just to see class imbalance bacth wise....
labels = np.unique(Y)
labels

array(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.',
       'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.',
       'neptune.', 'nmap.', 'normal.', 'perl.', 'phf.', 'pod.',
       'portsweep.', 'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.',
       'warezclient.', 'warezmaster.'], dtype=object)

### Batch class imbalance reweighting

In [22]:
nuiq,cnt = np.unique(Y_type, return_counts = True)
maxval = max(cnt)

X_new = X.copy()
Y_new = Y.copy()

for i in range(nuiq.shape[0]):
    
    Ytypedd = Y[Y_type == nuiq[i]]
    
    Xdd = X[Y_type == nuiq[i]]
    
    times = int(500/Ytypedd.shape[0])

#     print(times)
#     print(np.tile(Xdd, (times,1)).shape, X.shape)
    
    X_new = np.concatenate([X_new , np.tile(Xdd, (times,1))], axis = 0)
    Y_new = np.concatenate([Y_new , np.tile(Ytypedd, times)], axis = 0)
    
    Ytypedd = None
    Xdd =None
    gc.collect()
    
X = X_new
Y = Y_new

In [23]:
X.shape

(839810, 41)

In [81]:
cw_num = class_weight.compute_class_weight('balanced',np.unique(Y_num),Y_num)
cw_cat = class_weight.compute_class_weight('balanced',np.unique(Y_cat),Y_cat)
cw_type = class_weight.compute_class_weight('balanced',np.unique(Y_type),Y_type)
cws_num = np.arange(2)*0.0
cws_num[np.unique(Y_num).astype(int)] = cw_num
cws_cat = np.arange(5)*0.0
cws_cat[np.unique(Y_cat).astype(int)] = cw_cat
cws_type = np.arange(23)*0.0
cws_type[np.unique(Y_type).astype(int)] = cw_type

In [25]:
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()

for i in label_list['type']:
    Y_cat[Y == i+str('.')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.')] = label_list[label_list['type'] == i]['id_type']

Y_num_hot = one_hot(Y_num.astype(int), 2)
Y_cat_hot = one_hot(Y_cat.astype(int), 5)
Y_type_hot = one_hot(Y_type.astype(int), 23)

X_new_hot = np.delete(X,[1,2,3],1)

X_new_hot2 = X[:,[1,2,3]]

In [26]:
model_pro_le = LE()
model_pro_ohe = OHE(handle_unknown='ignore')
model_srv_le = LE()
model_srv_ohe = OHE(handle_unknown='ignore')
model_flg_le = LE()
model_flg_ohe = OHE(handle_unknown='ignore')

model_pro_ohe.fit_transform(model_pro_le.fit_transform(cat_uniqs[0]).reshape(-1,1))
model_srv_ohe.fit_transform(model_srv_le.fit_transform(cat_uniqs[1]).reshape(-1,1))
model_flg_ohe.fit_transform(model_flg_le.fit_transform(cat_uniqs[2]).reshape(-1,1))

X_pro = model_pro_ohe.transform(model_pro_le.transform(X_new_hot2[:,0]).reshape(-1,1)).todense()
X_srv = model_srv_ohe.transform(model_srv_le.transform(X_new_hot2[:,1]).reshape(-1,1)).todense()
X_flg = model_flg_ohe.transform(model_flg_le.transform(X_new_hot2[:,2]).reshape(-1,1)).todense()

In [27]:
X_new = np.concatenate((X_new_hot , X_pro , X_srv, X_flg), axis = 1).astype(float)
X_new.shape

(839810, 122)

In [28]:
np.unique(Y_type)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22.])

In [29]:
np.unique(Y, return_counts = True)

(array(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.',
        'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.',
        'neptune.', 'nmap.', 'normal.', 'perl.', 'phf.', 'pod.',
        'portsweep.', 'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.',
        'warezclient.', 'warezmaster.'], dtype=object),
 array([   750,    504,    504,    532,    504,   2996,    507,    504,
           505, 201040,   1158, 616654,    501,    504,    501,   2779,
           505,   4081,   2397,    501,    712,    667,    504]))

In [30]:
N,D = X_new.shape

In [31]:
Y.shape

(839810,)

## Modelling...

In [32]:
def get_model(input_shape):
    
    
    xin = kl.Input(shape = input_shape)
    x = kl.Dense(1024, activation='relu',kernel_regularizer=kr.l2(0.01))(xin)
#     x = kl.Dropout(rate = 0.3)(x)
    x = kl.Dense(128, activation='tanh',kernel_regularizer=kr.l2(0.1))(x)
#     x = kl.Dense(60, activation='tanh',kernel_regularizer=kr.l2(0.1))(x)
#     x = kl.Dense(64, activation='tanh',kernel_regularizer=kr.l2(0.1))(x)
#     x = kl.Dense(20, activation='tanh',kernel_regularizer=kr.l2(0.1))(x)
#     x = kl.Dense(16, activation='tanh',kernel_regularizer=kr.l2(0.1))(x)
    
#     x = kl.Dense(10, activation='tanh',kernel_regularizer=kr.l2(0.1))(x)
#     x = kl.Dropout(rate = 0.4)(x)
    x1 = kl.Dense(16, activation='tanh',kernel_regularizer=kr.l2(0.01))(x)
#     x1 = kl.Dropout(rate = 0.8)(x1)
    x1 = kl.Dense(8, activation='relu',kernel_regularizer=kr.l2(0.01))(x1)
    x1out = kl.Dense(2, activation='softmax', name = 'y1out',kernel_regularizer=kr.l2(0.01))(x1)
    
#     x2cat = kl.Concatenate(axis=-1)([x1out, x])
    
    x2 = kl.Dense(32, activation='tanh',kernel_regularizer=kr.l2(0.01))(x)
#     x2 = kl.Dropout(rate = 0.8)(x2)
    x2 = kl.Dense(16, activation='relu',kernel_regularizer=kr.l2(0.01))(x2)
    
    x2out = kl.Dense(5, activation='softmax', name = 'y2out',kernel_regularizer=kr.l2(0.01))(x2)
    
#     x3cat = kl.Concatenate(axis=-1)([x2out, x])
    
    x3 = kl.Dense(64, activation='tanh',kernel_regularizer=kr.l2(0.01))(x)
#     x3 = kl.Dropout(rate = 0.8)(x3)
    x3 = kl.Dense(32, activation='relu',kernel_regularizer=kr.l2(0.01))(x3)
    
    x3out = kl.Dense(23, activation='softmax', name = 'y3out',kernel_regularizer=kr.l2(0.01))(x3)
    
    model = km.Model(
        inputs=xin,
        outputs=[x1out,x2out,x3out],
        name="fashionnet")
    return model

In [33]:
model = get_model([D])

In [120]:
# losses = {   
#     "y1out": "categorical_crossentropy",
#     "y2out": "categorical_crossentropy",
#     "y3out": "categorical_crossentropy",
# }
losses = {
    "y1out": "categorical_hinge",
    "y2out": "categorical_hinge",
    "y3out": "categorical_hinge",
}

lossWeights = {"y1out": 23.0, "y2out": 5.0, "y3out": 2.0}

opt = ko.Nadam()
# opt = ko.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)

model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights, metrics=["accuracy"])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           (None, 122)          0                                            
__________________________________________________________________________________________________
dense_371 (Dense)               (None, 1024)         125952      input_76[0][0]                   
__________________________________________________________________________________________________
dense_372 (Dense)               (None, 128)          131200      dense_371[0][0]                  
__________________________________________________________________________________________________
dense_373 (Dense)               (None, 16)           2064        dense_372[0][0]                  
__________________________________________________________________________________________________
dense_375 

In [39]:
model.output

[<tf.Tensor 'y1out/Softmax:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'y2out/Softmax:0' shape=(?, 5) dtype=float32>,
 <tf.Tensor 'y3out/Softmax:0' shape=(?, 23) dtype=float32>]

### Loading pretrained model...

In [40]:
model = km.load_model('model.h5')

### Training model


> Dont need to train if you loaded pretrained model...


In [82]:
# sample training...single epoch
train = True
if(train):
    for i in range(1):
        clr()
        print(i)
        his = model.fit(X_new,{"y1out": Y_num_hot, "y2out": Y_cat_hot, "y3out": Y_type_hot},epochs=3, batch_size = 5000, validation_split = 0.0
                  ,class_weight={"y1out": cws_num, "y2out": cws_cat, "y3out": cws_type})

0
Epoch 1/3
318323/318323 [==============================] - 17s 54us/step - loss: 0.9141 - y1out_loss: 0.0126 - y2out_loss: 0.0356 - y3out_loss: 0.0407 - y1out_acc: 0.9939 - y2out_acc: 0.9817 - y3out_acc: 0.9792
Epoch 2/3
318323/318323 [==============================] - 17s 54us/step - loss: 0.9625 - y1out_loss: 0.0142 - y2out_loss: 0.0359 - y3out_loss: 0.0412 - y1out_acc: 0.9931 - y2out_acc: 0.9818 - y3out_acc: 0.9792
Epoch 3/3
318323/318323 [==============================] - 18s 55us/step - loss: 0.9053 - y1out_loss: 0.0127 - y2out_loss: 0.0349 - y3out_loss: 0.0411 - y1out_acc: 0.9939 - y2out_acc: 0.9825 - y3out_acc: 0.9792


In [83]:
### Saving trained model last time....
model.save('model.h5')
# model.save('model_best.h5')

# Checking on test data split....  25 %

In [84]:
testfile = 'test_red.npy'
# Sample batching....
batcharr = np.load(testfile)
batcharr.shape

(318323, 42)

In [85]:
X = batcharr[:,:-1]

Y = batcharr[:,-1]
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()

for i in label_list['type']:

    Y_cat[Y == i + "."] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+ "."] = label_list[label_list['type'] == i]['id_type']


In [86]:
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()

for i in label_list['type']:
    Y_cat[Y == i+str('.')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.')] = label_list[label_list['type'] == i]['id_type']

Y_num_hot = one_hot(Y_num.astype(int), 2)
Y_cat_hot = one_hot(Y_cat.astype(int), 5)
Y_type_hot = one_hot(Y_type.astype(int), 23)

X_new_hot = np.delete(X,[1,2,3],1)

X_new_hot2 = X[:,[1,2,3]]

# X_new.shape

model_pro_le = LE()
model_pro_ohe = OHE(handle_unknown='ignore')
model_srv_le = LE()
model_srv_ohe = OHE(handle_unknown='ignore')
model_flg_le = LE()
model_flg_ohe = OHE(handle_unknown='ignore')

model_pro_ohe.fit_transform(model_pro_le.fit_transform(cat_uniqs[0]).reshape(-1,1))
model_srv_ohe.fit_transform(model_srv_le.fit_transform(cat_uniqs[1]).reshape(-1,1))
model_flg_ohe.fit_transform(model_flg_le.fit_transform(cat_uniqs[2]).reshape(-1,1))

X_pro = model_pro_ohe.transform(model_pro_le.transform(X_new_hot2[:,0]).reshape(-1,1)).todense()
X_srv = model_srv_ohe.transform(model_srv_le.transform(X_new_hot2[:,1]).reshape(-1,1)).todense()
X_flg = model_flg_ohe.transform(model_flg_le.transform(X_new_hot2[:,2]).reshape(-1,1)).todense()

X_srv.shape

X_new = np.concatenate((X_new_hot , X_pro , X_srv, X_flg), axis = 1).astype(float)

X_new.shape

(318323, 122)

In [87]:
lables, _ = np.unique(Y, return_counts = True)
lables, _

(array(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'normal.', 'perl.', 'phf.', 'pod.', 'portsweep.',
        'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'], dtype=object),
 array([   305,      7,      1,     16,      4,   1331,      8,      4,      2,  98058,    412, 213737,      1,      1,     57,    975,      5,   1703,   1195,      1,    231,    260,      9]))

In [88]:
lossacc = model.evaluate(X_new,{"y1out": Y_num_hot, "y2out": Y_cat_hot, "y3out": Y_type_hot}, batch_size = 1000)

318323/318323 [==============================] - 8s 26us/step


In [89]:
print('Final Test Losses, net_loss :',lossacc[0],', y1 :', lossacc[1], ', y2 :', lossacc[2], ', y3 :', lossacc[3])
print('Final Test Accuracies, y1 :', lossacc[4], ', y2 :', lossacc[5], ', y3 :', lossacc[6])

Final Test Losses, net_loss : 0.8377100313033332 , y1 : 0.010496585979162759 , y2 : 0.03530940625020098 , y3 : 0.04080048163304848
Final Test Accuracies, y1 : 0.9951244506072934 , y2 : 0.9818423424289341 , y3 : 0.9792255042932262


In [90]:
from sklearn.metrics import confusion_matrix as cmat
from sklearn.metrics import precision_recall_fscore_support as prfs

In [91]:
Y_preds = model.predict(X_new, batch_size = 1000)

In [92]:
mat1 = cmat(np.argmax(Y_num_hot, axis = 1), np.argmax(Y_preds[0], axis = 1))
mat2 = cmat(np.argmax(Y_cat_hot, axis = 1), np.argmax(Y_preds[1], axis = 1))
mat3 = cmat(np.argmax(Y_type_hot, axis = 1), np.argmax(Y_preds[2], axis = 1))

In [93]:
np.set_printoptions(linewidth = 200)

In [94]:
print(mat1)

[[213490    247]
 [  1305 103281]]


In [95]:
print(mat2)

[[213654     83      0      0      0]
 [   965  98889      0      0      0]
 [  1861   2560      0      0      0]
 [   292      2      0      0      0]
 [    17      0      0      0      0]]


In [96]:
print(mat3)

[[213727      0      0      0      0      0      0      0      0      0     10      0      0      0      0      0      0      0      0      0      0      0      0]
 [   305      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     7      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     1      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [    16      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     2      0      0      0      0      0      0      0      0      0      2      0      0      0      0      0      0      0      0      0      0      0      0]
 [  1229      0 

In [97]:
prfsarr = np.array(prfs(Y, labels[np.argmax(Y_preds[2], axis = 1)], labels = labels))
df = pd.DataFrame(prfsarr.T, columns = ['precison', 'recall', 'f_score', 'support'], index = labels)

/home/abhikcr/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [99]:
df.mean()

precison        0.000089
recall          0.050232
f_score         0.000177
support     13840.130435
dtype: float64

In [119]:
his.history

{'loss': [0.914133483202602, 0.96245527657901, 0.905327709105584],
 'y1out_loss': [0.012634130023786836,
  0.014200420275543349,
  0.012699805783171663],
 'y2out_loss': [0.03556606910464466,
  0.03590912790976767,
  0.034896042881448416],
 'y3out_loss': [0.040732239565508185,
  0.04117930016213266,
  0.04106077331969526],
 'y1out_acc': [0.9939055552114794, 0.9930919213206872, 0.9938898501220528],
 'y2out_acc': [0.9816884065911847, 0.9818077830236391, 0.9824580718005345],
 'y3out_acc': [0.9791720989529977, 0.9791626719604016, 0.9791563886530822]}

### Thankyou for viewing